#### Task 1.1
Write Python and SQL code to create a DB named Contractor.db and tables for the data model in the CA2 Sub-Contractor Problem. All the necessary database contraints must be implemented.

In [1]:
import sqlite3
con = sqlite3.connect("Contractor.db")
sql_skill = '''
CREATE TABLE Skill (
SkillID INTEGER,
SkillName TEXT,
CostPerHour REAL,
PRIMARY KEY ( SkillID AUTOINCREMENT)
) '''
sql_employee ='''
CREATE TABLE Employee (
EmployeeID INTEGER,
EmployeeName TEXT,
SkillID INTEGER,
PRIMARY KEY (EmployeeID AUTOINCREMENT),
FOREIGN KEY (SkillID) REFERENCES Skill(SkillID)
) 
'''

sql_project = '''
CREATE TABLE Project (
ProjectID INTEGER,
ProjectTitle TEXT,
StartDate DATE,
EndDate DATE,
PRIMARY KEY ( ProjectID AUTOINCREMENT)
)  '''
sql_project_employee = '''
CREATE TABLE ProjectEmployee (
ProjectID INTEGER,
EmployeeID INTEGER,
HoursWork INTEGER,
PRIMARY KEY (ProjectID, EmployeeID),
FOREIGN KEY (ProjectID) REFERENCES Project (ProjectID),
FOREIGN KEY (EmployeeID) REFERENCES Employee (EmployeeID)
)  '''
try:
    con.execute("BEGIN")
    con.execute(sql_skill)
    con.execute(sql_employee)
    con.execute(sql_project)
    con.execute(sql_project_employee)
    con.commit()
except Exception as ex:
    print(ex)
    con.rollback()
    
con.close()

#### Task 1.2
The  spreadsheet use by the company has been exported to the file Project.csv. Write Python  and SQL code to store each item of data in the correct place in the database.

In [ ]:
import sqlite3, csv
data = list(csv.reader(open("Projects.csv")))
con = sqlite3.connect("Contractor.db")


for row in data[1:]:
    ## Skill Table
    skill_sql = '''INSERT INTO Skill VALUES (?,?,?)'''
    try:
        con.execute(skill_sql, row[6:9])
    except sqlite3.DatabaseError as ex:
        print(ex)
    
    employee_sql = '''INSERT INTO Employee VALUES (?,?,?)'''
    try:
        con.execute(employee_sql, row[4:7])
    except sqlite3.DatabaseError as ex:
        print(ex)

    project_sql = '''INSERT INTO Project VALUES (?,?,?,?)'''
    try:
        con.execute(project_sql, row[:4])
    except sqlite3.DatabaseError as ex:
        print(ex)
    p_e_sql = '''INSERT INTO ProjectEmployee VALUES (?,?,?)'''
    try:
        con.execute(p_e_sql, [row[0],row[4],row[9]])
    except sqlite3.DatabaseError as ex:
        print(ex)
con.commit()
con.close()

Write a Python program and the necessary files to create a web application as follows:
-   The form.html is provided and will be used as the landing or home page for your web app, i.e. this page will be the first page to be rendered when the web browser access your web app using http://127.0.0.1:5000/
-   The  code in form.html has already been written, such that after a user has entered the value of the Project ID and click submit, the form data will be posted to the url "/project".
-   Write code to retrieve all the employees' names and the hours worked on the project with the Project ID entered in the form.

The html output should look like this:

## Project ID: 1
| Employee Name | Hours Worked |
| -- | -- |
| Smith | 45 |
| Jones | 30 |
| Roberts | 12 |


In [5]:
from flask import Flask, render_template, redirect, request
import sqlite3
app = Flask("Mr Flask")


@app.route("/") # [landing page]
def root():
    return render_template("form.html") #[use of jinja template]


@app.route("/project", methods=["POST","GET"])
def query_project():
    if request.method == "GET":
        return '''Not Supported 
        <a href="/>Home</a>'''
    try:
        con = sqlite3.connect("Contractor.db")
        project_id = request.form["project_id"] #[retrieve data from form]
        sql = '''
SELECT EmployeeName, HoursWork FROM ProjectEmployee
INNER JOIN Employee ON ProjectEmployee.EmployeeID = Employee.EmployeeID
WHERE ProjectEmployee.ProjectID = ?
        ''' #[sql]
        cur = con.execute(sql, [project_id])
        data = cur.fetchall() #[ retrieve data]
        con.close()
        return render_template("index.html",
                               project_id = project_id,
                               results=data)
        #[use of jinja template]
    except sqlite3.DatabaseError as ex:
        print(ex)
        con.close()

app.run()

 * Serving Flask app 'Mr Flask' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Sep/2024 12:39:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2024 12:39:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Sep/2024 12:39:49] "POST /project HTTP/1.1" 200 -
